In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

## Mens Section

In [ ]:
mens_section=[
    ['https://www.flipkart.com/clothing-and-accessories/topwear/pr?sid=clo%2Cash&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Top+wear&page=',2018,'Topwear'],
    ['https://www.flipkart.com/clothing-and-accessories/bottomwear/pr?sid=clo%2Cvua&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Bottom+wear&page=',2013,'Bottomwear'],
    ['https://www.flipkart.com/clothing-and-accessories/blazers-suits-waistcoat-coat/pr?sid=clo%2Cupk&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Suits%2C+Blazers+%26+Waistcoats&page=',271,'Blazers, Waistcoats And Suits'],
    ['https://www.flipkart.com/clothing-and-accessories/clothing-accessories/pr?sid=clo%2Cqd8&p%5B%5D=facets.ideal_for%255B%255D%3DMen&p%5B%5D=facets.ideal_for%255B%255D%3Dmen&p%5B%5D=facets.ideal_for%255B%255D%3Dmen&otracker=categorytree&otracker=nmenu_sub_Men_0_Ties%2C+Socks%2C+Caps+%26+More&page=',1018,'Clothing Accessories'],
    ['https://www.flipkart.com/clothing-and-accessories/fabric/pr?sid=clo%2Cqfi&p%5B%5D=facets.ideal_for%255B%255D%3DMen&p%5B%5D=facets.ideal_for%255B%255D%3Dmen&p%5B%5D=facets.ideal_for%255B%255D%3Dmen&otracker=categorytree&otracker=nmenu_sub_Men_0_Fabrics&page=',996,'Fabric'],
    ['https://www.flipkart.com/clothing-and-accessories/winter-wear/pr?sid=clo%2Cqvw&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Winter+Wear&page=',2002,'Winterwear'],
    ['https://www.flipkart.com/clothing-and-accessories/ethnic-wear/pr?sid=clo%2Ccfv&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Ethnic+wear&page=',2002,'Ethnic'],
    ['https://www.flipkart.com/clothing-and-accessories/~cs-axj382hyd1/pr?sid=clo&collection-tab-name=Innerwear+And+Loungewear&otracker=nmenu_sub_Men_0_Innerwear+%26+Loungewear&page=',1753,'InnerWear and Loungewear'],
    ['https://www.flipkart.com/clothing-and-accessories/~cs-4ayq68p3ip/pr?sid=clo&collection-tab-name=Raincoats+And+Windcheaters&otracker=nmenu_sub_Men_0_Raincoats+%26+Windcheaters&page=',56,'Raincoats And Windcheaters'],
    ['https://www.flipkart.com/watches/wrist-watches/pr?sid=r18%2Cf13&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&page=',2003,'Wrist Watch'],
    ['https://www.flipkart.com/search?sid=reh&otracker=CLP_Filters&p%5B%5D=facets.ideal_for%255B%255D%3DMen&page=',2010,'Bags, Wallets & Belts']
]

In [81]:
men_link =[]
for i in mens_section:
    for j in range(1,i[1]):
        result = requests.get(i[0]+str(j),headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")
        s=soup.find_all('div',{'class':'_13oc-S'})
        for m in s:
            n=m.find_all('a')
            for k in n:
                men_link.append("https://www.flipkart.com"+k.attrs['href'])       

In [82]:
men_link=list(set(men_link))

In [122]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [123]:
err=[]
for i in tqdm(men_link[:20]):
    try:
        result = requests.get(i,headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")
        product_name=soup.find('span',{'class':'B_NuCI'}).text
        product_brand=soup.find('span',{'class':'G6XhRU'}).text
        product_link=i
        product_cat=soup.find_all('div',{'class':'_3GIHBu'})[3].text
        try:
            product_des=soup.find('div',{'class':'_1AN87F'}).text
        except:
            product_des=soup.find('div',{'class':'row _1v8OXw'}).text
        product_mrp=soup.find('div',{'class':'_30jeq3 _16Jk6d'}).text
        product_mrp=int(''.join(product_mrp[1:].split(',')))
        product_price=soup.find('div',{'class':'_3I9_wc _2p6lqe'}).text
        product_price=int(''.join(product_price[1:].split(',')))
        images=[]
        li=soup.find_all('div',{'class':'q6DClP _2_B7hD'})
        for i in li:
            images.append('https://rukminim1.flixcart.com/image/714/857/'+i.attrs['style'].split('128/128/')[1][:-1])
        primary_link=images[0]
        secondary_link=','.join(images[1:])
        s_chart=soup.find_all('a',{'class':'_1fGeJ5 _2UVyXR _31hAvz'})
        size=[]
        for i in s_chart:
            size.append(i.text)
        size_avl=','.join(size)
    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
   
    
    

100%|██████████| 20/20 [00:47<00:00,  2.37s/it]


In [124]:
men_data=pd.DataFrame({
    
    "Website":'https://www.flipkart.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [136]:
conn = sqlite3.connect(r'flipkart.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
conn.commit()

men_data.to_sql('product_details', conn, if_exists='replace', index = False)
conn.close()

## Womens Section

In [127]:
women_section=[
    ['https://www.flipkart.com/clothing-and-accessories/pr?sid=clo&marketplace=FLIPKART&p%5B%5D=facets.ideal_for%255B%255D%3DWomen&otracker=nmenu_sub_Women_0_Clothing&page=',2284,'Clothing And Accessories'],
    ['https://www.flipkart.com/womens-footwear/pr?sid=osp%2Ciko&otracker=nmenu_sub_Women_0_Footwear&page=',2016,'Womens Footwear'],
    ['https://www.flipkart.com/search?sid=r18&otracker=CLP_Filters&p%5B%5D=facets.ideal_for%255B%255D%3DWomen&page=',2002,'Watches'],
    ['https://www.flipkart.com/wearable-smart-devices/smart-watches/pr?sid=ajy,buh&p%5B%5D=facets.filter_standard%255B%255D%3D1&facets.availability[]=Exclude+Out+of+Stock&otracker=CLP_lhs&otracker=nmenu_sub_Women_0_Smart%20Watches&page=',1677,'Smart Watches'],
    ['https://www.flipkart.com/jewellery/pr?sid=mcr&otracker=nmenu_sub_Women_0_Jewellery&page=',2213,'Jewellery'],
    ['https://www.flipkart.com/search?sid=reh%2Fihu&otracker=CLP_Filters&p%5B%5D=facets.ideal_for%255B%255D%3DGirls&p%5B%5D=facets.ideal_for%255B%255D%3DWomen&page=',2005,'Handbags & Clutches']
]

In [128]:
women_link =[]
for i in women_section[0:1]:
    for j in range(1,10):
        result = requests.get(i[0]+str(j),headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")
        s=soup.find_all('div',{'class':'_13oc-S'})
        for m in s:
            n=m.find_all('a')
            for k in n:
                women_link.append("https://www.flipkart.com"+k.attrs['href'])       

In [129]:
women_link=list(set(women_link))

In [134]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [135]:
err=[]
for i in tqdm(women_link[:20]):
    try:
        result = requests.get(i,headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")
        product_name=soup.find('span',{'class':'B_NuCI'}).text
        product_brand=soup.find('span',{'class':'G6XhRU'}).text
        product_link=i
        product_cat=soup.find_all('div',{'class':'_3GIHBu'})[3].text
        try:
            product_des=soup.find('div',{'class':'_1AN87F'}).text
        except:
            product_des=soup.find('div',{'class':'row _1v8OXw'}).text
        product_mrp=soup.find('div',{'class':'_30jeq3 _16Jk6d'}).text
        product_mrp=int(''.join(product_mrp[1:].split(',')))
        product_price=soup.find('div',{'class':'_3I9_wc _2p6lqe'}).text
        product_price=int(''.join(product_price[1:].split(',')))
        images=[]
        li=soup.find_all('div',{'class':'q6DClP _2_B7hD'})
        for i in li:
            images.append('https://rukminim1.flixcart.com/image/714/857/'+i.attrs['style'].split('128/128/')[1][:-1])
        primary_link=images[0]
        secondary_link=','.join(images[1:])
        s_chart=soup.find_all('a',{'class':'_1fGeJ5 _2UVyXR _31hAvz'})
        size=[]
        for i in s_chart:
            size.append(i.text)
        size_avl=','.join(size)
    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
   
    
    

100%|██████████| 20/20 [00:29<00:00,  1.47s/it]


In [137]:
women_data=pd.DataFrame({
    
    "Website":'https://www.flipkart.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [138]:
women_data

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://www.flipkart.com/,https://www.flipkart.com/leelavati-printed-dai...,Printed Daily Wear Crepe Saree (Gold),Leelavati,Sarees,,999,345,Women,Style CodeVS-593-1PatternPrintedPack of1Occasi...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
1,https://www.flipkart.com/,https://www.flipkart.com/koisa-girls-bra-free-...,Women Full Coverage Lightly Padded Bra (Multi...,KOISA,Bras,"28,30,32,34,Regular,Lightly Padded,Styled Back,No",1000,308,Women,Good quality and comfortable bra for you. It i...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
2,https://www.flipkart.com/,https://www.flipkart.com/jil-x-women-hipster-m...,Women Hipster Multicolor Panty (Pack of 10),Jil X,Panties,"S,M,L",799,417,Women,Suitable ForEthnic WearStyle CodeJILPRIE10Wais...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
3,https://www.flipkart.com/,https://www.flipkart.com/nightblue-checkered-f...,Checkered Fashion Cotton Silk Saree (Beige),NightBlue,Sarees,,1999,299,Women,Style CodeNBS1175-(2)PatternCheckeredPack of1S...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
4,https://www.flipkart.com/,https://www.flipkart.com/lewania-women-girls-t...,Women T-Shirt Lightly Padded Bra (Multicolor),LEWANIA,Bras,"28,30,32,34A,36A,38,40,Regular,Lightly Padded,...",1799,424,Women,This is comfortable and stylish. Wear it under...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
5,https://www.flipkart.com/,https://www.flipkart.com/montrez-full-sleeve-s...,Full Sleeve Solid Women Denim Jacket,MONTREZ,Jackets,"S,M,L,XL,XXL",1499,399,Women,ColorDark BlueFabricDenimPatternSolidStyle cod...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
6,https://www.flipkart.com/,https://www.flipkart.com/clovia-women-full-cov...,Women Full Coverage Non Padded Bra (Grey),Clovia,Bras,"32C,32D,38B,Regular,Non Padded,Regular,No",599,279,Women,Product Type: Full Coverage Bra ; Fabric Type:...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
7,https://www.flipkart.com/,https://www.flipkart.com/kriska-women-embroide...,Women Embroidered Rayon A-line Kurta (Maroon),Kriska,Kurtas,"S,M,L,XL,XXL",1999,539,Women,KRISKA CREATION is a women’s ethnic clothing b...,https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
8,https://www.flipkart.com/,https://www.flipkart.com/fashion-ritmo-floral-...,"Floral Print, Printed, Geometric Print Daily W...",Fashion Ritmo,Sarees,,699,299,Women,"This Saree Launce By Fashion Ritmo,Printed Sar...",https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,
9,https://www.flipkart.com/,https://www.flipkart.com/harpa-women-a-line-pi...,Women A-line Pink Dress,HARPA,Dresses,"S,M,L,XL",2199,676,Women,"Pink Printed Woven A-line Midi Dress, has a Ro...",https://rukminim1.flixcart.com/image/714/857/k...,https://rukminim1.flixcart.com/image/714/857/k...,


In [139]:
conn = sqlite3.connect(r'flipkart.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()

women_data.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()